<a href="https://colab.research.google.com/github/CSID-DGU/2021-1-CECD4-Hashtag-3/blob/main/YOLOv4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. clone darknet

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
!/usr/local/cuda/bin/nvcc --version

In [ ]:
!make

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

뒤에서 이미지를 출력하고 upload하고 download할 때 사용할 함수 선언

In [ ]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

In [ ]:
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights data/person.jpg

In [ ]:
imShow('predictions.jpg')

내 로컬 메모리에 있는 파일을 업로드 하는 부분

In [ ]:
from google.colab import files
file_uploaded = files.upload()

위 과정에서 업로드한 test.png 파일을 이용해 욜로 실행

In [ ]:
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights ../test.jpg
imShow('predictions.jpg')

In [ ]:
%cd darknet/

내 드라이브 연동

In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

내 드라이브를  /mydrive라는 루트로 접근할 수 있도록 설정

In [ ]:
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
%cd darknet

내 드라이브에 있는 이미지 파일을 이용해서 임계값이  0.5  이상인 경우를 고려해 출력

In [ ]:
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights /mydrive/images/cats.jpg -thresh 0.5 -ext_output -dont_show 
imShow('predictions.jpg')

predictions.jpg 파일을 내 드라이브에 images파일에 detection1.jpg 파일로 저장

In [ ]:
!cp predictions.jpg /mydrive/images/detection1.jpg

처리하고자 하는 이미지 목록이 적혀있는 images.txt파일을 이용해 이미지 처리를 진행한 후 result.json 파일을 결과로 출력

In [ ]:
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights -ext_output -dont_show -out result.json < /mydrive/images.txt

train

1. 새로 학습시킬 데이터 생성 ( 1class - 1000 )
- 구글을 통해 이미지 다운로드(https://github.com/theAIGuysCode/Download-Google-Images)
- labelImg를 통해 라벨링 진행(https://github.com/tzutalin/labelImg)

2. 만들어진  dataset 에서 70-80% obj.zip으로 만들고 20-30% test.zip으로 만든 후 내 google 드라이브에 업로드

내 드라이브 yolov4 폴더에 있는 파일들 출력

In [ ]:
!ls /mydrive/yolov4

내 드라이브에 있는 파일 카피

In [ ]:
!cp /mydrive/yolov4/obj.zip ../
!cp /mydrive/yolov4/test.zip  ../

In [ ]:
!unzip ../obj.zip -d data/
!unzip ../text.zip -d data/

3. configuring files for training

1) cfg file

In [ ]:
!cp cfg/yolov4-custom.cfg /mydrive/yolov4/yolov4-obj.cfg   

In [ ]:
download('cfg/yolov4-custom.cfg')
# 다운로드 후 로컬에서 yolov4-obj.cfg로 이름 바꾸기

- batch=64, subdivisions =16 권장 ( subdivisions = 32 도 가능 )
- width , height = 32 or 16 으로 설정하는게 일반적 ( 더 크면 느려짐 )
- max_batches = (# of classes ; 2) * 3000 ( 적어도 6000이 되야함. )
- steps = 0.8 * max_batches = 0.8 * 6000 = 4800/ 0.9 * 6000 = 5400
- filters = (# of classes ; 2 + 5 ) * 3 = 21
=> max_batches = 6000 / steps = 4800, 5400 / classes = 2/ filters = 21
####* 메모리 사용 문제가 있는 경우 cfg파일에서 random = 1 을 random = 0 으로 변경

In [ ]:
!cp /mydrive/yolov4/yolov4-obj.cfg ./cfg

2) obj.names and obj.data

- classes.txt 파일과 동일한 순서로 한줄에 하나씩 class 입력한 obj.names 파일 생성(스페이스 쓰면 안됨.)
- classes =2, train = data.train.txt, valid = data/test.txt, names = data/obj.names , backup = /mydrive/yolov4/backup( 폴더 생성해야 함 ) 


In [ ]:
!cp /mydrive/yolov4/obj.names ./data
!cp /mydrive/yolov4/obj.data ./data

3)  train.txt and test.txt 파일 생성 
-  https://github.com/theAIGuysCode/YOLOv4-Cloud-Tutorial/tree/master/yolov4 링크에서 generate_test.py 와 generate_train.py 복사 후 업로드

In [ ]:
!cp /mydrive/yolov4/generate_trian.py ./
!cp /mydrive/yolov4/generate_test.py ./

In [ ]:
!python generate_train.py
!python generate_test.py

In [ ]:
!ls data/

4) 학습에 이용할 pre_trained weights 파일 다운로드

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

5) 학습 시작!

In [ ]:
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg yolov4.conv.137 -dont_show -map

-  학습 완료 후 차트 출력

In [ ]:
imShow('chart.png')

- 중간에 끝나는 경우 가장 마지막에 출력된 weight파일을 기준으로 시작 

In [ ]:
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_last.weights -dont_show

- 학습 끝 -> 학습 시킨 파일로 욜로 실행

In [ ]:
%cd cfg
!sed -i 's/batch=64/batch=1' yolov4-obj.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov4-obj.cfg
%cd ..

In [ ]:
!./darknet detector test data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_last.weights /mydrive/images/car2.jpg -thresh 0.3
imShow('predictions.jpg')